In [2]:
!pip install umap-learn

In [3]:
!pip install seaborn==0.11.2

In [4]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import os
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Paths for the containers that we will create
# S3 Bucket where the data is stored
BUCKET_NAME = "tbilisimeetup"  # Put your S3 bucket name here
BUCKET = f's3://{BUCKET_NAME}'

# data paths
DATA_FOLDER = 'data'
DATA_FILE = 'data.csv'
DATA_PATH = os.path.join(BUCKET, DATA_FOLDER, DATA_FILE)


df = pd.read_csv(DATA_PATH)

NameError: name 'RAW_DATA_PATH' is not defined